# Handcrafted Fully Connected Neural Net (FCNN)

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

### Standardizing the Data 

In [3]:
# The model was tested with both standardization, and normalization. Normalization doesn't seem to 
# give good results, that's why we finalized our design with standardization. 
# standardize the training data
from sklearn.preprocessing import StandardScaler
scale= StandardScaler()

Xtr_standardized = scale.fit_transform(Xtr) # revise this line as needed
Xts_standardized = scale.fit_transform(Xts) # revise this line as needed
ytr_standardized = ytr # revise this line as needed

Xtr_torch = torch.Tensor(Xtr_standardized)
Xts_torch = torch.Tensor(Xts_standardized)
ytr_torch = torch.Tensor(ytr_standardized)

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr_standardized, delimiter=",")

### Train Test Split, and Creation of Pytorch Dataloaders 

In [4]:
x_train,x_test,y_train,y_test = train_test_split(Xtr_torch,ytr_torch,test_size=0.2,random_state = random.randint(0,1000))
train_ds = TensorDataset(x_train, y_train)
test_ds =  TensorDataset(x_test, y_test)
batch_size = 32
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)


### Handcrafted Model CNN

In [43]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=1,out_channels=32, kernel_size=2, stride=1, padding=1),
#             nn.BatchNorm1d(5),
            nn.Dropout(0.2),
            nn.Sigmoid())
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=10, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm1d(10),
            nn.Sigmoid(), 
            nn.MaxPool1d(kernel_size = 1, stride = 1))
        self.fc = nn.Sequential(
#             nn.Dropout(0.5),
            nn.Linear(90, 90),
            nn.Sigmoid())
        self.fc1 = nn.Sequential(
            nn.Linear(90, 85),
#             nn.Dropout(0.5),
            nn.Sigmoid())
        self.fc2= nn.Sequential(
            nn.Linear(85, 85),
            nn.Sigmoid())
        self.fc3= nn.Sequential(
            nn.Linear(85, 1),
            nn.Sigmoid())
    def forward(self, x):
#         print(x.shape)
        out = self.layer1(x)
#         print("Layer1: ",out.shape)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
#         print("Layer2: ",out.shape)
        out = self.fc(out)
#         print("FC1: ",out.shape)
        out = self.fc1(out)
#         print("FC2: ",out.shape)
        out = self.fc2(out)
#         print("FC3: ",out.shape)
        out = self.fc3(out)
        return out
model_vgg = VGG16(num_classes=1)
print(str(model_vgg))

VGG16(
  (layer1): Sequential(
    (0): Conv1d(1, 32, kernel_size=(2,), stride=(1,), padding=(1,))
    (1): Dropout(p=0.2, inplace=False)
    (2): Sigmoid()
  )
  (layer2): Sequential(
    (0): Conv1d(32, 10, kernel_size=(1,), stride=(1,))
    (1): Sigmoid()
    (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=90, out_features=90, bias=True)
    (1): Sigmoid()
  )
  (fc1): Sequential(
    (0): Linear(in_features=90, out_features=85, bias=True)
    (1): Sigmoid()
  )
  (fc2): Sequential(
    (0): Linear(in_features=85, out_features=85, bias=True)
    (1): Sigmoid()
  )
  (fc3): Sequential(
    (0): Linear(in_features=85, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [46]:
model_vgg = VGG16(num_classes=1)
x_train,x_test,y_train,y_test = train_test_split(Xtr_torch,ytr_torch,test_size=0.2,random_state = 42)
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1).permute(0,2,1)
y_train = y_train.reshape(y_train.shape[0],1,1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1).permute(0,2,1)
y_test = y_test.reshape(y_test.shape[0],1,1)

train_ds = TensorDataset(x_train, y_train)
test_ds =  TensorDataset(x_test, y_test)
batch_size = 100
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

# num_epoch = 10
lrate = 2e-3
decay = lrate/num_epoch

opt = optim.RMSprop(model_vgg.parameters(), lr=lrate)

# lambda1 = lambda epoch: (1-decay)*epoch
# scheduler = optim.lr_scheduler.LambdaLR(opt, lr_lambda=lambda1)

criterion = nn.BCELoss()


In [47]:
# TODO
import time


num_epoch = 100

a_tr_loss = np.zeros([num_epoch])
a_tr_accuracy = np.zeros([num_epoch])
a_ts_loss = np.zeros([num_epoch])
a_ts_accuracy = np.zeros([num_epoch])

for epoch in range(num_epoch):
    correct = 0 # initialize error counter
    total = 0 # initialize total counter
    batch_loss = []
    tic = time.time()
    
    model_vgg.train() # put model in training mode
    # iterate over training set
    for train_iter, data in enumerate(train_dl):
        x_batch,y_batch = data
        y_batch = y_batch.view(-1,1)
        out = model_vgg(x_batch)
#         print("y_batch ",y_batch)
        # Compute Loss
#         print(y_batch.shape)
        loss = criterion(out,y_batch.type(torch.float))
        batch_loss.append(loss.item())
        # Zero gradients
        opt.zero_grad()
        # Compute gradients using back propagation
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        predicted = out.clamp(0,1).round().type(torch.long)
#         print("predicted: ",predicted)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
    a_tr_loss[epoch] = np.mean(batch_loss) # Compute average loss over epoch
    a_tr_accuracy[epoch] = 100*correct/total
    
    correct = 0
    total = 0
    batch_loss = []
    model_vgg.eval() # put model in evaluation mode
    with torch.no_grad():
        for data in test_dl:
            images, labels = data
            labels = labels.view(-1,1)
            out = model_vgg(images)
            batch_loss.append(criterion(out,labels.type(torch.float)).item())
            predicted = out.clamp(0,1).round().type(torch.long)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    a_ts_loss[epoch] = np.mean(batch_loss)
    a_ts_accuracy[epoch] = 100*correct/total
    elapsed = time.time()-tic
    
    # Print details every print_mod epoch
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
          +'Train Accuracy: {0:.2f}    Test Loss: {1:.3f}   '.format(a_tr_accuracy[epoch], a_ts_loss[epoch])
          +'Test Accuracy: {0:.2f}   '.format(a_ts_accuracy[epoch])
          +'Time: {0:.2f}sec'.format(elapsed))

Epoch:  1   Train Loss: 0.651   Train Accuracy: 65.33    Test Loss: 0.632   Test Accuracy: 63.40   Time: 0.47sec
Epoch:  2   Train Loss: 0.502   Train Accuracy: 76.91    Test Loss: 0.474   Test Accuracy: 79.50   Time: 0.47sec
Epoch:  3   Train Loss: 0.452   Train Accuracy: 80.15    Test Loss: 0.457   Test Accuracy: 80.05   Time: 0.46sec
Epoch:  4   Train Loss: 0.441   Train Accuracy: 80.70    Test Loss: 0.443   Test Accuracy: 80.85   Time: 0.46sec
Epoch:  5   Train Loss: 0.438   Train Accuracy: 80.79    Test Loss: 0.474   Test Accuracy: 78.80   Time: 0.49sec
Epoch:  6   Train Loss: 0.425   Train Accuracy: 81.61    Test Loss: 0.453   Test Accuracy: 79.75   Time: 0.46sec
Epoch:  7   Train Loss: 0.420   Train Accuracy: 81.83    Test Loss: 0.428   Test Accuracy: 82.25   Time: 0.44sec
Epoch:  8   Train Loss: 0.417   Train Accuracy: 82.08    Test Loss: 0.427   Test Accuracy: 81.85   Time: 0.46sec
Epoch:  9   Train Loss: 0.412   Train Accuracy: 82.29    Test Loss: 0.426   Test Accuracy: 81.60

Epoch: 74   Train Loss: 0.329   Train Accuracy: 86.14    Test Loss: 0.355   Test Accuracy: 85.55   Time: 1.33sec
Epoch: 75   Train Loss: 0.329   Train Accuracy: 85.86    Test Loss: 0.357   Test Accuracy: 85.40   Time: 0.91sec
Epoch: 76   Train Loss: 0.329   Train Accuracy: 86.14    Test Loss: 0.354   Test Accuracy: 85.60   Time: 0.67sec
Epoch: 77   Train Loss: 0.327   Train Accuracy: 86.38    Test Loss: 0.355   Test Accuracy: 85.65   Time: 0.64sec
Epoch: 78   Train Loss: 0.329   Train Accuracy: 86.11    Test Loss: 0.358   Test Accuracy: 85.30   Time: 0.62sec
Epoch: 79   Train Loss: 0.330   Train Accuracy: 86.22    Test Loss: 0.353   Test Accuracy: 85.80   Time: 0.65sec
Epoch: 80   Train Loss: 0.329   Train Accuracy: 86.24    Test Loss: 0.363   Test Accuracy: 84.85   Time: 0.60sec
Epoch: 81   Train Loss: 0.326   Train Accuracy: 86.49    Test Loss: 0.360   Test Accuracy: 85.10   Time: 0.82sec
Epoch: 82   Train Loss: 0.327   Train Accuracy: 86.09    Test Loss: 0.361   Test Accuracy: 85.15

In [53]:
# generate kaggle submission file using the validation script
# !python {"validation.py " + model_savepath + " --Xts_path " + Xts_savepath + " --Xtr_path " + Xtr_savepath + " --yts_hat_path " + yts_hat_savepath } 

training auc =  0.9199856429850293
test label confidences saved in yts_hat_pytorch.csv


This model when uploaded to kaggle showed bad AUC score. This is because performance was degraded when CNN was tried on this dataset